### Import necessary packages

In [15]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Read JSON File

In [20]:
with open("headline_sarcasm/Sarcasm_Headlines_Dataset.json", "r") as f:
    sentences = []
    labels = []
    url = []
    
    for line in f:
        data = json.loads(json.loads(json.dumps([line]))[0])
        url.append(data['article_link'])
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])

### Tokenization

In [26]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index=tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'to': 2, 'of': 3, 'the': 4, 'in': 5, 'for': 6, 'a': 7, 'on': 8, 'and': 9, 'with': 10, 'is': 11, 'new': 12, 'trump': 13, 'man': 14, 'from': 15, 'at': 16, 'about': 17, 'you': 18, 'this': 19, 'by': 20, 'after': 21, 'up': 22, 'out': 23, 'be': 24, 'how': 25, 'as': 26, 'it': 27, 'that': 28, 'not': 29, 'are': 30, 'your': 31, 'his': 32, 'what': 33, 'he': 34, 'all': 35, 'just': 36, 'who': 37, 'has': 38, 'will': 39, 'more': 40, 'one': 41, 'into': 42, 'report': 43, 'year': 44, 'why': 45, 'have': 46, 'area': 47, 'over': 48, 'donald': 49, 'u': 50, 'day': 51, 'says': 52, 's': 53, 'can': 54, 'first': 55, 'woman': 56, 'time': 57, 'like': 58, 'her': 59, "trump's": 60, 'old': 61, 'no': 62, 'get': 63, 'off': 64, 'an': 65, 'life': 66, 'people': 67, 'obama': 68, 'now': 69, 'house': 70, 'still': 71, "'": 72, 'women': 73, 'make': 74, 'was': 75, 'than': 76, 'white': 77, 'back': 78, 'my': 79, 'i': 80, 'clinton': 81, 'down': 82, 'if': 83, '5': 84, 'when': 85, 'world': 86, 'could': 87, 'we': 88, 'th

### Sequences

In [27]:
sequence = tokenizer.texts_to_sequences(sentences)
print(sequence)

[[308, 15115, 679, 3337, 2298, 48, 382, 2576, 15116, 6, 2577, 8434], [4, 8435, 3338, 2746, 22, 2, 166, 8436, 416, 3112, 6, 258, 9, 1002], [145, 838, 2, 907, 1749, 2093, 582, 4719, 221, 143, 39, 46, 2, 10736], [1485, 36, 224, 400, 2, 1832, 29, 319, 22, 10, 2924, 1393, 6969, 968], [767, 719, 4720, 908, 10737, 623, 594, 5, 4, 95, 1309, 92], [10738, 4, 365, 73], [4, 6970, 351, 6, 461, 4274, 2195, 1486], [19, 479, 39, 1168, 31, 155, 2, 99, 83, 18, 158, 6, 32, 352], [249, 3623, 6971, 555, 5274, 1995, 141], [2094, 326, 347, 401, 60, 15117, 6, 4, 3896], [2925, 1680, 4721, 14, 37, 4275, 6972, 5, 2095, 1103], [286, 782, 462, 8, 1556, 1911, 9, 3624], [234, 514, 2926, 13, 9, 929, 226, 369, 2, 4276, 15118, 8437], [238, 3897, 8438, 3339, 38, 235, 15119, 6, 7, 173], [15120, 1394, 665, 651, 5, 327, 3, 1031], [534, 2095, 10739, 123, 10740, 6, 10741, 4722, 1912], [2578, 1395, 383, 45, 3898, 348, 319, 1032, 2, 24, 15121, 20, 1104, 387, 103, 1310], [1681, 8439, 3113, 8440, 20, 6973, 1218], [10742, 857, 2,